Find weight adjustment for imbalanced dataset for BCEWithLogitsLoss


https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

In [ ]:
import torch as ch
import pandas as pd
import os
from prediction.short_term_outcome_prediction.timeseries_decomposition import decompose_and_label_timeseries

In [ ]:
data_path = '/Users/jk1/temp/opsum_end/preprocessing/gsu_Extraction_20220815_prepro_08062024_083500/early_neurological_deterioration_train_data_splits/train_data_splits_early_neurological_deterioration_ts0.8_rs42_ns5.pth'

In [ ]:
use_gpu = ch.cuda.is_available()
target_time_to_outcome = 6

In [ ]:
splits = ch.load(os.path.join(data_path))

In [ ]:
# recommended weight adjustement
recommended_weight_adjustment_df = pd.DataFrame(columns=['split', 'train', 'val'])
for idx, split in enumerate(splits):
    X_train, X_val, y_train, y_val = split
    train_map, train_flat_labels = decompose_and_label_timeseries(X_train, y_train, target_time_to_outcome=target_time_to_outcome)
    val_map, val_flat_labels = decompose_and_label_timeseries(X_val, y_val, target_time_to_outcome=target_time_to_outcome)
    
    n_pos_train = sum(train_flat_labels) + 1e-6
    n_neg_train = len(train_flat_labels) - n_pos_train
    n_pos_val = sum(val_flat_labels) + 1e-6
    n_neg_val = len(val_flat_labels) - n_pos_val

    print(f'Weight adjustment for train: {n_neg_train/n_pos_train}')
    print(f'Weight adjustment for val: {n_neg_val/n_pos_val}')
    
    recommended_weight_adjustment_df = pd.concat([recommended_weight_adjustment_df, pd.DataFrame([[idx, n_neg_train/n_pos_train, n_neg_val/n_pos_val]], columns=['split', 'train', 'val'])], ignore_index=True)

In [ ]:
recommended_weight_adjustment_df

In [ ]:
overall_average = recommended_weight_adjustment_df[['train', 'val']].median()
overall_average.median()